In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [4]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/200"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 60 # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import timm

# Create model
model = timm.create_model("hf_hub:timm/regnety_160.tv2_in1k", pretrained=True)
model.to(device)  # Move model to GPU

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.1 MB/s eta 0:00:00
Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Epoch 1/10, Train Loss: 4.8368, Train Accuracy: 27.11%, Val Loss: 2.4740, Val Accuracy: 49.17%


...

Epoch 6/10, Train Loss: 0.1168, Train Accuracy: 95.95%, Val Loss: 0.3504, Val Accuracy: 87.25%


Epoch 7/10, Train Loss: 0.0892, Train Accuracy: 96.81%, Val Loss: 0.3370, Val Accuracy: 88.08%


Epoch 8/10, Train Loss: 0.0793, Train Accuracy: 97.37%, Val Loss: 0.3228, Val Accuracy: 88.74%


Epoch 9/10, Train Loss: 0.0528, Train Accuracy: 98.08%, Val Loss: 0.3188, Val Accuracy: 89.24%


Epoch 10/10, Train Loss: 0.0490, Train Accuracy: 98.15%, Val Loss: 0.3219, Val Accuracy: 89.40%
Test Accuracy: 89.40%


In [5]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      1.000     0.917     0.957        36
          PT      0.815     0.710     0.759        31
          PC      0.957     0.882     0.918        51
          MC      0.855     0.908     0.881        65
          LC      0.691     0.792     0.738        48
           F      0.867     0.889     0.878        81
          DC      0.934     0.927     0.931       260
           A      0.938     0.938     0.938        32

    accuracy                          0.894       604
   macro avg      0.882     0.870     0.875       604
weighted avg      0.897     0.894     0.895       604



In [6]:
# Train the model
num_epochs = 20
for epoch in range(10, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 11/20, Train Loss: 0.0504, Train Accuracy: 97.87%, Val Loss: 0.3027, Val Accuracy: 89.40%


Epoch 12/20, Train Loss: 0.0508, Train Accuracy: 98.23%, Val Loss: 0.3156, Val Accuracy: 89.74%


Epoch 13/20, Train Loss: 0.0606, Train Accuracy: 97.66%, Val Loss: 0.2605, Val Accuracy: 90.73%


Epoch 14/20, Train Loss: 0.0547, Train Accuracy: 97.66%, Val Loss: 0.2850, Val Accuracy: 89.24%


Epoch 15/20, Train Loss: 0.0427, Train Accuracy: 98.08%, Val Loss: 0.2814, Val Accuracy: 89.40%


Epoch 16/20, Train Loss: 0.0339, Train Accuracy: 98.23%, Val Loss: 0.2953, Val Accuracy: 89.57%


Epoch 17/20, Train Loss: 0.0474, Train Accuracy: 97.94%, Val Loss: 0.3170, Val Accuracy: 88.91%
Epoch 00017: reducing learning rate of group 0 to 1.0000e-05.


Epoch 18/20, Train Loss: 0.0361, Train Accuracy: 98.51%, Val Loss: 0.2994, Val Accuracy: 89.07%


Epoch 19/20, Train Loss: 0.0265, Train Accuracy: 99.08%, Val Loss: 0.2950, Val Accuracy: 89.74%


Epoch 20/20, Train Loss: 0.0266, Train Accuracy: 98.94%, Val Loss: 0.2855, Val Accuracy: 90.56%
Test Accuracy: 90.56%


In [7]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      1.000     0.889     0.941        36
          PT      1.000     0.774     0.873        31
          PC      0.958     0.902     0.929        51
          MC      0.918     0.862     0.889        65
          LC      0.704     0.792     0.745        48
           F      0.875     0.951     0.911        81
          DC      0.917     0.935     0.926       260
           A      0.969     0.969     0.969        32

    accuracy                          0.906       604
   macro avg      0.918     0.884     0.898       604
weighted avg      0.910     0.906     0.906       604



In [8]:
# Train the model
num_epochs = 30
for epoch in range(20, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 21/30, Train Loss: 0.0234, Train Accuracy: 99.01%, Val Loss: 0.2854, Val Accuracy: 91.06%
Epoch 00021: reducing learning rate of group 0 to 1.0000e-06.


Epoch 22/30, Train Loss: 0.0266, Train Accuracy: 99.01%, Val Loss: 0.2780, Val Accuracy: 90.07%


Epoch 23/30, Train Loss: 0.0327, Train Accuracy: 98.58%, Val Loss: 0.2747, Val Accuracy: 90.89%


Epoch 24/30, Train Loss: 0.0331, Train Accuracy: 98.44%, Val Loss: 0.2836, Val Accuracy: 90.23%


Epoch 25/30, Train Loss: 0.0344, Train Accuracy: 98.44%, Val Loss: 0.2777, Val Accuracy: 90.07%
Epoch 00025: reducing learning rate of group 0 to 1.0000e-07.


Epoch 26/30, Train Loss: 0.0269, Train Accuracy: 98.72%, Val Loss: 0.2768, Val Accuracy: 90.89%


Epoch 27/30, Train Loss: 0.0266, Train Accuracy: 98.58%, Val Loss: 0.2749, Val Accuracy: 90.73%


Epoch 28/30, Train Loss: 0.0307, Train Accuracy: 98.65%, Val Loss: 0.2777, Val Accuracy: 90.89%


Epoch 29/30, Train Loss: 0.0279, Train Accuracy: 98.94%, Val Loss: 0.2743, Val Accuracy: 90.73%
Epoch 00029: reducing learning rate of group 0 to 1.0000e-08.


Epoch 30/30, Train Loss: 0.0264, Train Accuracy: 98.65%, Val Loss: 0.2774, Val Accuracy: 90.89%
Test Accuracy: 90.89%


In [9]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      1.000     0.917     0.957        36
          PT      1.000     0.774     0.873        31
          PC      0.958     0.902     0.929        51
          MC      0.903     0.862     0.882        65
          LC      0.709     0.812     0.757        48
           F      0.885     0.951     0.917        81
          DC      0.924     0.935     0.929       260
           A      0.969     0.969     0.969        32

    accuracy                          0.909       604
   macro avg      0.919     0.890     0.902       604
weighted avg      0.913     0.909     0.910       604

